# Ewaluacja jednoklasowych algorytmów klasyfikacji w danych strumieniowych



## Wstęp

Materiały do listy został zainspirowany tutorialem udostępnionych przez twórców biblioteki dotyczący klasyfikacji danych strumieniowych w niezbalansowanym ustawieniu [River - imbalanced learning tutorial](https://riverml.xyz/0.14.0/examples/imbalanced-learning/). 

Autorzy w swoich materiałach wykorzystują zbiór danych `CreditCard`, w którym zawarto informacje o oszustwach związanych z kartami kredytowymi. Zapoznajmy się teraz ze zbiorem danych.

### Przedstawienie zbioru danych

In [ ]:
from river import datasets

ds = datasets.CreditCard()
ds

Jak można wywnioskować z opisu transakcje opisane jako oszustwa stanowią jedynie **0.172%** zbioru, a dokładnie `492` z `284807` transakcji. Cechy V1-V28 są cechami otrzymanymi poprzez wykorzystanie transformacji PCA, ze względu na poufność danych transakcyjnych. `Time` oznacza czas pomiędzy każdą z transakcji a pierwszą oraz `Amount` która określa wartość transakcji. Wczytajmy teraz pierwszy rekord.

In [ ]:
from IPython.display import display_pretty

display_pretty(list(ds.take(1)))

### Motywacja

Autorzy materiałów co prawda przedstawiają rozwiązanie problemu oraz przedstawiają techniki radzenia sobie z niezbalansowanymi danymi, ale popełnili błędy w swoim układzie eksperymentalnym:

**1) Autorzy nie uwzględnili specyfiki problemu przewidywania oszustw w transakcjach przeprowadzonych z wykorzystaniem kart płatniczych.**

Autorzy stworzyli układ eksperymentalny w taki sposób, że model jest aktualizowany z pojawianiem się każdej transakcji. W przypadku tego problemu, w rzeczywistym systemie nie moglibyśmy wykorzystać takiego scenariusza, dlatego, że informacja dotycząca czy dana transakcja była oszustwem przychodzi ze sporym opóźnieniem. 
Sprawdzając regulamin jednego z operatorów transakcji, znajdujemy informacje, że konsument ma nawet `120` dni na złożenie reklamacji, a procedura może wynieść nawet kilkadziesiąt kolejnych dni.

**2) Autorzy wykorzystali metrykę ROCAUC do zmierzenia jakości algorytmu danych niezbalansowanych**

Metryka ROCAUC nie jest metryką stworzoną do obsługi takiego scenariusza, w danych niezbalansowanych powinno stosować się metrykę [Precesion Recall Curve](https://scikit-learn.org/stable/auto_examples/model_selection/plot_precision_recall.html). Więcej o powodach korzystania z PRC można przeczytać m.in. w następującym artykule https://towardsdatascience.com/imbalanced-data-stop-using-roc-auc-and-use-auprc-instead-46af4910a494

Dodatkowo sprawdzimy przydatność cechy `Time`

## Klasyfikacja danych strumieniowych w bibliotece river

### Wstęp 

Rozważmy na początku kod który przedstawiają autorzy

In [ ]:
from river import linear_model
from river import preprocessing
from river import evaluate
from river import metrics

model = preprocessing.StandardScaler() | linear_model.LogisticRegression()

ds = datasets.CreditCard()
auc = metrics.ROCAUC()
evaluate.progressive_val_score(ds, model, auc)
auc

Tak jak wspomniane zostało wcześniej powyższa ewaluacja nie uwzględnia opóźnień w etykietach. Jednakże trzeba mieć na uwadze, że metoda [`progressive_val_score`](`https://riverml.xyz/0.14.0/api/evaluate/progressive-val-score/) zawiera parametr `delay`, który odpowiada za ten scenariusz. Jednakże w przypadku tego zbioru nie jesteśmy tego przetestować, ze względu na brak informacji w zbiorze. Stworzymy własny alternatywny scenariusz.

### Poprawa scenariuszu ewaluacyjnego

Poprawmy teraz scenariusz ewaluacji:   
 - Pierwsze 200000 transakcji wykorzystamy w celu treningu modelu;
 - Pozostałe rekordy wykorzystamy do inferencji modelu i do przeliczenia metryk;
 - Niestety biblioteka `river` nie posiada implementacji wspomnianej wcześniej metryki `PRC`, dlatego wykorzystamy metrykę `Macro F1`

```{hint}

Przy przetwarzaniu danych strumieniowych i danych temporalnych często stosuje się podział przez konkretne daty zamiast polegać na liczbie rekordów.
```

Niestety obiekt typu `river.datasets.base.Dataset` nie pozwala na manipulacje zbiorem, dlatego stworzymy własny iterator, w którym musimy dodatkowo wykonać konwersję cech. W celach porównawczych stworzymy dwie wersje iteratora jedną zawierającą cechę `Time` i drugą bez tej cechy.

In [ ]:
from collections.abc import Iterator
from typing import Any

from river.stream import iter_csv


def get_ds_iter() -> Iterator[Any]:
    return iter_csv(
        ds.path,
        converters={
            "Amount": float,
            "Class": int,
            **{f"V{i}": float for i in range(1, 29)},
        },
        drop=["Time"],
        target="Class",
    )


def get_ds_iter_with_time() -> Iterator[Any]:
    return iter_csv(
        ds.path,
        converters={
            "Amount": float,
            "Class": int,
            "Time": float,
            **{f"V{i}": float for i in range(1, 29)},
        },
        target="Class",
    )

#### Uczenie modelu

In [ ]:
from tqdm.notebook import tqdm
from river.metrics import MacroF1

ds_iter = get_ds_iter()
train_split_idx = 200_000
model = preprocessing.StandardScaler() | linear_model.LogisticRegression()
f1_metric = MacroF1()

pbar = tqdm(desc="Training", total=train_split_idx)
for idx, (x, y) in enumerate(ds_iter):
    pbar.update(1)

    y_pred = model.predict_one(x)
    f1_metric = f1_metric.update(y_true=y, y_pred=y_pred)
    model = model.learn_one(x, y)
    if idx + 1 == train_split_idx:
        pbar.close()
        break


f1_metric

#### Ewaluacja modelu

Pozostaje nam dokonać ewaluacji danych testowych. Podczas inferencji możemy dodatkowo wykorzystać metodę [`compose.pure_inference_mode`](https://riverml.xyz/0.14.0/api/compose/pure-inference-mode/), która zapewnia że żaden z elementów naszego pipelinu nie zostanie zaaktualizowany i zostanie wykorzystana czysta inferencja. Wykorzystajmy ją w naszym scenariuszu.

In [ ]:
from river import compose

test_f1_metric = MacroF1()
with compose.pure_inference_mode():
    for idx, (x, y) in tqdm(
        enumerate(ds_iter),
        desc="Inference",
    ):
        y_pred = model.predict_one(x)
        test_f1_metric = test_f1_metric.update(y_true=y, y_pred=y_pred)

test_f1_metric

#### Refactor funkcjonalności uczenia i ewaluacji do klasy `RiverTrainer`

Opakujmy teraz to w jedną klasę aby poprawić czytelność kodu do dalszych porównań. W celach porównawczych dodano metrykę AUC oraz uzupełniono metody o elementy potrzebne do analiz. 

In [ ]:
from dataclasses import dataclass, field
from typing import Generator, Any, List, Optional, Union

from river.compose.pipeline import Pipeline
from river.metrics.base import Metric, Metrics
from river.metrics import ROCAUC
from tqdm.notebook import tqdm


@dataclass
class RiverTrainer:
    pipeline: Pipeline
    data_iter: Iterator[Any]
    train_split_idx: Optional[int]
    train_metrics: Union[Metric, Metrics]
    eval_metrics: Optional[Union[Metric, Metrics]] = None
    auc: bool = False
    train_auc_metric: Optional[ROCAUC] = field(init=False, default=None)
    eval_auc_metric: Optional[ROCAUC] = field(init=False, default=None)

    def __post_init__(self) -> None:
        if self.auc:
            self.train_auc_metric = ROCAUC()
            if train_split_idx:
                self.eval_auc_metric = ROCAUC()

    def train(self) -> None:
        if self.train_split_idx:
            pbar = tqdm(desc="Training", total=train_split_idx)
        else:
            pbar = tqdm(desc="Training")
        for idx, (x, y) in enumerate(self.data_iter):
            pbar.update(1)

            y_pred = self.pipeline.predict_one(x)
            if self.train_auc_metric:
                y_proba = self.pipeline.predict_proba_one(x)
                self.train_auc_metric = self.train_auc_metric.update(
                    y_true=y, y_pred=y_proba
                )

            if self.train_metrics:
                self.train_metrics = self.train_metrics.update(y_true=y, y_pred=y_pred)
            self.pipeline = self.pipeline.learn_one(x, y)
            if self.train_split_idx and idx + 1 == self.train_split_idx:
                break

    def test(self) -> None:
        with compose.pure_inference_mode():
            for idx, (x, y) in tqdm(
                enumerate(self.data_iter),
                desc="Inference",
            ):
                y_pred = self.pipeline.predict_one(x)
                if self.eval_auc_metric:
                    y_proba = self.pipeline.predict_proba_one(x)
                    self.eval_auc_metric = self.eval_auc_metric.update(
                        y_true=y, y_pred=y_proba
                    )
                if self.eval_metrics:
                    self.eval_metrics = self.eval_metrics.update(
                        y_true=y, y_pred=y_pred
                    )

    def evaluate(self) -> None:
        self.train()
        if self.train_split_idx:
            self.test()

#### Uczenie modelu z wykorzystaniem naszego `RiverTrainera`

In [ ]:
trainer = RiverTrainer(
    pipeline=(preprocessing.StandardScaler() | linear_model.LogisticRegression()),
    data_iter=get_ds_iter(),
    train_split_idx=200_000,
    train_metrics=MacroF1(),
    eval_metrics=MacroF1(),
    auc=False,
)
trainer.evaluate()

In [ ]:
print("Train", trainer.train_metrics, "\nTest", trainer.eval_metrics)

#### Test poprawności `RiverTrainer`

Porówanie obiektu `RiverTrainer` z metodą `progressive_val_score`. W tym celu stworzymy nowy iterator zawierający zmienną `Time`

```{warning}
Scenariusz wyłącznie w celu zweryfikowania poprawności kodu
```

In [ ]:
model = preprocessing.StandardScaler() | linear_model.LogisticRegression()

river_auc = metrics.ROCAUC()
evaluate.progressive_val_score(ds, model, river_auc)


trainer = RiverTrainer(
    pipeline=(preprocessing.StandardScaler() | linear_model.LogisticRegression()),
    data_iter=get_ds_iter_with_time(),
    train_split_idx=None,
    train_metrics=None,
    eval_metrics=None,
    auc=True,
)
trainer.evaluate()

print("River evaluation", river_auc, "Custom evaluation", trainer.train_auc_metric)

### Analizy

In [ ]:
results = {}

#### Model bez uwzględnienia cechy `Time`

In [ ]:
trainer = RiverTrainer(
    pipeline=(preprocessing.StandardScaler() | linear_model.LogisticRegression()),
    data_iter=get_ds_iter(),
    train_split_idx=200_000,
    train_metrics=MacroF1(),
    eval_metrics=MacroF1(),
    auc=False,
)
trainer.evaluate()
results["Without Time Feature"] = {
    "train": trainer.train_metrics.get(),
    "test": trainer.eval_metrics.get(),
}

#### Model z uwzględnieniem cechy `Time`

In [ ]:
trainer = RiverTrainer(
    pipeline=(preprocessing.StandardScaler() | linear_model.LogisticRegression()),
    data_iter=get_ds_iter_with_time(),
    train_split_idx=200_000,
    train_metrics=MacroF1(),
    eval_metrics=MacroF1(),
    auc=False,
)
trainer.evaluate()
results["With Time Feature"] = {
    "train": trainer.train_metrics.get(),
    "test": trainer.eval_metrics.get(),
}

#### Model uczony na pełnych danych (sklearn)

In [ ]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import f1_score

df_ds = pd.read_csv(ds.path)
train_ds = df_ds[:200_000]
test_ds = df_ds[200_000:]
y_train = train_ds.pop("Class")
y_test = test_ds.pop("Class")

pipeline = Pipeline(
    [("scaler", StandardScaler()), ("logistic_regression", LogisticRegression())]
).fit(train_ds, y_train)

train_y_pred = pipeline.predict(train_ds)
test_y_pred = pipeline.predict(test_ds)

results["Sklearn model"] = {
    "train": f1_score(y_true=y_train, y_pred=train_y_pred, average="macro"),
    "test": f1_score(y_true=y_test, y_pred=test_y_pred, average="macro"),
}

#### Rezultaty

In [ ]:
pd.DataFrame(results)

#### Podsumowanie wyników

Na pierwszy rzut oka można byłoby dojść do następujących wniosków:   

**1.** Logistyczna regresja uczona w trybie online ma dużo wyższą skuteczność niż jej które uczy na całości danych   
**2.** Zastosowanie cechy `Time`, która określa upływ czasu od pierwszego zdarzenia polepsza jakość klasyfikacji
   
Ale czy aby na pewno?

**Ad 1.** Podczas uczenia modeli pomiędzy bibliotekami skorzystano z innych optymalizatorów i hiperparametrów. Aby uczciwie odpowiedzieć na pytanie czy logistyczna rogresja uczona w trybie online rzeczywiście osiąge wyniki musielibyśmy użyć tych samych hiperparametrów metod i dopiero wtedy je ze sobą porównać. Jedyną różnicą powinien być jedynie tryb uczenia

**Ad 2.** Sprawdźmy z jakiego okresu pochodzą dane

In [ ]:
import numpy as np

times = []
for x, y in ds:
    times.append(x["Time"])

print("Seconds elapsed", np.max(times), "\nHours elapsed", np.max(times) / 3600)

Dane pochodzą jedynie z dwóch dni, co może być za krótkim okresem żeby potwierdzić użyteczność tej zmiennej. Model może wykorzystać informacje o różnicach rozkładów pomiędzy dwoma dniami, co w tym przypadku może być uznane jako pewnego rodzaju artefakt.